# Analyzing data with MongoDB and Dask

## Sample AirBnB Listings Dataset

In this notebook, we will use the [sample AirBnB listings dataset](https://docs.atlas.mongodb.com/sample-data/sample-airbnb/) provided by MongoDB, and hosted on a free-tier cluster on Mongo Atlas. For information on how to load this dataset into your cluster check this [link](https://docs.atlas.mongodb.com/sample-data/#std-label-load-sample-data).

## Read data using `dask-mongo`

- Use `read_mongo` to get the dataset into a Dask Bag
- Filter data using normal Dask operations
- Convert from a Bag to a Dask DataFrame
- Perform a groupby operation

In [1]:
from dask_mongo import read_mongo, to_mongo
from dask.distributed import Client

client = Client()
client

Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57552,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:57559,Total threads: 2
Dashboard: http://127.0.0.1:57561/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:57555,


In [2]:
# Replace this for your URI connection
host_uri = "mongodb+srv://<username>:<password>@<cluster-address>/myFirstDatabase?retryWrites=true&w=majority"

b = read_mongo(
    connection_kwargs={"host": host_uri},
    database="sample_airbnb",
    collection="listingsAndReviews",
    chunksize=500,
)

Let's take a look at the first record of our data set

In [3]:
# Uncomment to take a look
# b.take(1)

In [4]:
b.pluck("property_type").frequencies().compute()

[('House', 606),
 ('Apartment', 3626),
 ('Condominium', 399),
 ('Loft', 142),
 ('Guesthouse', 50),
 ('Hostel', 34),
 ('Serviced apartment', 185),
 ('Bed and breakfast', 69),
 ('Treehouse', 1),
 ('Bungalow', 14),
 ('Guest suite', 81),
 ('Townhouse', 108),
 ('Villa', 32),
 ('Cabin', 15),
 ('Other', 18),
 ('Chalet', 2),
 ('Farm stay', 9),
 ('Boutique hotel', 53),
 ('Boat', 2),
 ('Cottage', 20),
 ('Earth house', 1),
 ('Aparthotel', 23),
 ('Resort', 11),
 ('Tiny house', 7),
 ('Nature lodge', 2),
 ('Hotel', 26),
 ('Casa particular (Cuba)', 9),
 ('Barn', 1),
 ('Hut', 1),
 ('Camper/RV', 2),
 ('Heritage hotel (India)', 1),
 ('Pension (South Korea)', 1),
 ('Campsite', 1),
 ('Houseboat', 1),
 ('Castle', 1),
 ('Train', 1)]

### Filtered and flattened data for DataFrame friendly shape

Since this is a rich, unstructured dataset, let's filter some useful information and get it into a structured Dask DataFrame. We will flatten down this data to be able to use common pandas operations. 

In [5]:
def process(record):
    try:
        yield {
            "accomodates": record["accommodates"],
            "bedrooms": record["bedrooms"],
            "price": float(str(record["price"])),
            "country": record["address"]["country"],
        }
    except KeyError:
        pass

In [6]:
# Filter only apartments 
b_flattened = (
    b.filter(lambda record: record["property_type"] == "Apartment")
    .map(process)
    .flatten()
)

In [7]:
b_flattened.take(3)

({'accomodates': 4, 'bedrooms': 1, 'price': 317.0, 'country': 'Brazil'},
 {'accomodates': 1, 'bedrooms': 1, 'price': 40.0, 'country': 'United States'},
 {'accomodates': 2, 'bedrooms': 1, 'price': 701.0, 'country': 'Brazil'})

We can now convert this Bag into a DataFrame using `to_dataframe` and perform some operations. 

In [8]:
ddf = b_flattened.to_dataframe()
ddf.head()

,accomodates,bedrooms,price,country
0,4,1,317.0,Brazil
1,1,1,40.0,United States
2,2,1,701.0,Brazil
3,2,1,135.0,United States
4,4,1,119.0,Brazil


### Groupy operation

With our nicely formatted Dask DataFrame, we can perform all the usual DataFrame operations we're used to. Let's compute the average price per country via a groupby operation.

In [9]:
ddf.groupby(["country"])["price"].mean().compute()

country
Australia        168.174174
Brazil           485.767033
Canada            84.860814
Hong Kong        684.622120
Portugal          66.112272
Spain             91.846442
Turkey           366.143552
United States    137.884228
China            448.300000
Name: price, dtype: float64

## Write to MongoDB using `dask-mongo`

- Convert Dask data frame to a Dask Bag
- Use `to_mongo` to write to the desired database

In this example we will convert the Dask DataFrame we just created and write it to a new database in our MongoDB Atlas cluster. 

In [10]:
import pandas as pd
import dask.bag as db

new_bag = ddf.to_bag(index=False, format='dict')

new_bag.take(1)

({'accomodates': 4, 'bedrooms': 1, 'price': 317.0, 'country': 'Brazil'},)

In [11]:
to_mongo(
    new_bag,
    database="new_database",
    collection="new_collection",
    connection_kwargs={"host": host_uri},
)